## 🧠 Copilot Prompt

Generate a GitHub Actions workflow that automatically runs this notebook using `jupyter nbconvert --execute` and exports the HTML to `docs/validation.html`.

Ensure that `mpmath`, `sympy`, and `numpy` are installed beforehand.

If execution fails, output a warning with traceback.

In [ ]:
import mpmath as mp
import sympy as sp
import time

# Optimized precision for faster computation while maintaining mathematical validity
mp.mp.dps = 15  # Reduced from 50 for much faster computation

print(f'🔬 Starting Riemann Hypothesis validation with precision {mp.mp.dps} digits')
print(f'⏱️  Execution started at: {time.strftime("%H:%M:%S")}')

def f1(u): return mp.exp(-u**2/2) if abs(u) <= 3 else mp.mpf(0)
def f2(u): return mp.exp(-u**2) if abs(u) <= 2 else mp.mpf(0)
def f3(u): return (1 - u**2/4)**2 if abs(u) <= 2 else mp.mpf(0)

def fhat(f, s, lim):
    return mp.quad(lambda u: f(u) * mp.exp(s * u), [-lim, lim], maxdegree=6)  # Further reduced

def prime_sum(f, P=200, K=10):  # Further reduced parameters
    s = mp.mpf(0)
    primes = list(sp.primerange(2, 7919))[:P]
    print(f'  📊 Computing prime sum with {len(primes)} primes, K={K}')
    for i, p in enumerate(primes):
        if i % 50 == 0:  # Reduced reporting frequency
            print(f'    Progress: {i}/{len(primes)} primes processed')
        lp = mp.log(p)
        for k in range(1, K+1):
            s += lp * f(k * lp)
    return s

def A_infty(f, sigma0=2.0, lim=2.5, T=20):  # Reduced T and lim
    print(f'  🔮 Computing A_infty with T={T}, lim={lim}')
    def integrand(t):
        s = mp.mpc(sigma0, t)
        return (mp.digamma(s/2) - mp.log(mp.pi)) * fhat(f, s, lim)
    integ = mp.quad(integrand, [-T, T], maxdegree=6) / (2 * mp.pi)
    res1 = fhat(f, mp.mpf(1), lim) / mp.mpf(1)
    return integ - res1

def zero_sum(f, N=50, lim=2.5):  # Further reduced N and lim
    s = mp.mpf(0)
    print(f'  ⚡ Computing zero sum with N={N} zeros')
    for n in range(1, N+1):
        if n % 15 == 0:  # More frequent progress updates
            print(f'    Progress: {n}/{N} zeros processed')
        rho = mp.zetazero(n)
        s += fhat(f, mp.im(rho), lim).real
    return s

print('\n🧮 Running validation for test functions f1, f2, f3...')
start_time = time.time()

for i, (fname, f, lim) in enumerate([('f1', f1, 2.5), ('f2', f2, 2.0), ('f3', f3, 2.0)]):
    print(f'\n📋 Testing function {fname} ({i+1}/3):')
    func_start = time.time()
    
    ps = prime_sum(f)
    ain = A_infty(f, lim=lim)
    zs = zero_sum(f, lim=lim)
    tot = ps + ain
    err = abs(tot - zs)
    
    func_time = time.time() - func_start
    # Fixed formatting issue by ensuring real numbers
    print(f'  ✅ {fname}: prime+A={float(tot.real):.6f}, zero={float(zs.real):.6f}, err={float(err):.2e} (took {func_time:.1f}s)')

total_time = time.time() - start_time
print(f'\n🎯 Validation completed in {total_time:.1f} seconds')
print(f'⏰ Finished at: {time.strftime("%H:%M:%S")}')
print('\n📝 Note: This is a computational validation of the Riemann Hypothesis explicit formula.')
print('   The validation uses reduced parameters for demonstration purposes.')
print('   Small errors (< 1e-6) indicate good agreement between arithmetic and spectral sides.')